In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import os

from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import callbacks

from sklearn.model_selection import GroupKFold

In [2]:
class CFG:
    dataset_dir="../input/ranzcr-clip-catheter-line-classification/"
    models_dir="./models/"
    n_folds=10
    target_cols=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present']

In [3]:
train=pd.read_csv(f"{CFG.dataset_dir}train.csv")

In [5]:
def get_fold(train):
    fold=train.copy()
    splitter=GroupKFold(n_splits=CFG.n_folds)
    for n,(train_idx,val_idx) in enumerate(splitter.split(train,groups=train["PatientID"])):
        fold.loc[val_idx,"folds"]=n
    fold["folds"]=fold["folds"].astype(int)
    return fold

fold=get_fold(train)

In [6]:
npz=np.load("../input/effnet_tuned_output.npz")
features_list=[npz[uid] for uid in tqdm(train["StudyInstanceUID"])]
features=np.array(features_list)

100%|██████████| 30083/30083 [00:55<00:00, 544.08it/s]


In [7]:
n=0
train_idx=(fold["folds"]!=n)
val_idx=(fold["folds"]==n)

X_train=features[train_idx]
X_val=features[val_idx]
y_train=train.loc[train_idx,CFG.target_cols]
y_val=train.loc[val_idx,CFG.target_cols]

In [8]:
def create_model():
    dense_model=keras.Sequential([
        keras.layers.Dense(len(CFG.target_cols),input_shape=(2560,),activation="sigmoid")
    ])
    adam=keras.optimizers.Adam(learning_rate=1e-4)
    dense_model.compile(optimizer=adam,loss="binary_crossentropy",metrics=[keras.metrics.AUC(multi_label=True,name="auc")])
    return dense_model

In [13]:
ckpt_dir=f"{CFG.models_dir}effnet_tuned_dense/"
os.makedirs(ckpt_dir,exist_ok=True)

lr_reducer=callbacks.ReduceLROnPlateau(monitor="val_auc",patience=7,min_lr=1e-6,mode="max",factor=0.5,verbose=1)
checkpoint=callbacks.ModelCheckpoint(f"{ckpt_dir}ckpt",monitor="val_auc",mode="max",
    save_weights_only=True,save_best_only=True)

In [14]:
dense_model=create_model()

dense_model.fit(X_train,y_train,epochs=70,shuffle=True,validation_data=(X_val,y_val),callbacks=[lr_reducer,checkpoint])

best_model=dense_model.load_weights(f"{ckpt_dir}ckpt")
score=best_model.evaluate(X_val,y_val,verbose=0)
print(f"\nscore: {score}")

Epoch 1/100
847/847 [==============================] - 3s 3ms/step - loss: 0.2790 - auc: 0.8333 - val_loss: 0.1875 - val_auc: 0.9144
Epoch 2/100
847/847 [==============================] - 2s 3ms/step - loss: 0.1680 - auc: 0.9165 - val_loss: 0.1621 - val_auc: 0.9220
Epoch 3/100
847/847 [==============================] - 2s 3ms/step - loss: 0.1543 - auc: 0.9254 - val_loss: 0.1550 - val_auc: 0.9234
Epoch 4/100
447/847 [==============>...............] - ETA: 1s - loss: 0.1507 - auc: 0.9283

KeyboardInterrupt: 